## Generate 16 Variations of a single LUT Design

In [5]:
for i in range(1,16):
    output = f"{bin(i)[2:]}".zfill(16)
    # print(output)

    fh = open(f"./results/design_files/design_{i}.v","w+")
    # fh.write(f"{output}")
    fh.write(f"`timescale 1ns / 1ps\n")
    fh.write(f"\n")
    fh.write(f"module generic_func(lut_in3, lut_in2, lut_in1, lut_in0, lut_out);\n")
    fh.write(f"\n")
    # fh.write(f"input wire [3:0] lut_in;\n")
    for j in range(4):
        fh.write(f"input wire lut_in{j};\n")
    fh.write(f"output reg lut_out;\n")
    fh.write(f"\n")
    fh.write(f"always @(*) begin\n")
    fh.write(f"\tcase ({{lut_in3, lut_in2, lut_in1, lut_in0}})\n")
    for j in range(16):
        fh.write(f"\t\t4'b{bin(j)[2:].zfill(4)}: lut_out <= {output[j]};\n")
    fh.write(f"\t\tdefault: lut_out <= 0;\n")
    fh.write(f"\tendcase\n")
    fh.write(f"end\n")
    fh.write(f"endmodule\n")
    fh.close()

## OpenFPGA Build Commands
```
python3 openfpga_flow/scripts/run_fpga_task.py basic_tests/0_debug_task/bank_fpga
python3 openfpga_flow/scripts/run_fpga_task.py basic_tests/0_debug_task/frame_fpga
python3 openfpga_flow/scripts/run_fpga_task.py basic_tests/0_debug_task/cc_fpga
```

In [24]:
for i in range(16):
    print(f"bench{i}")

bench0
bench1
bench2
bench3
bench4
bench5
bench6
bench7
bench8
bench9
bench10
bench11
bench12
bench13
bench14
bench15


## Code to Generate 1,000 Randomized Designs

### Constraints 
- 2x2 CLB
- 16 LUTs (Random Number Instantiated)


In [46]:
import random
import os
import shutil

design_dir = "./results/random_designs"

shutil.rmtree(design_dir)
os.mkdir(design_dir)

class Wire:
    def __init__(self,name,lut,idx,dir):
        self.name = name
        self.lut = lut
        self.idx = idx
        self.dir = dir

for i in range(20000):
    # output = f"{bin(i)[2:]}".zfill(16)
    # print(output)

    index = i
    index_str = f"{(index + 1)}".zfill(5)
    
    os.mkdir(f"{design_dir}/{index_str}")
    fh = open(f"{design_dir}/{index_str}/design.v","w+")

    # NUM_LUTS = random.randint(1,16)
    NUM_LUTS = random.randint(1,16)
    NUM_LUT_INPUTS = 4
    # MAX_NUM_INPUTS = 5
    # NUM_INPUTS = random.randont(4,128-NUM_LUTS)
    # NUM_INPUTS = random.randint(4, 64 - NUM_LUTS)
    NUM_INPUTS = 4
    # NUM_OUTPUTS = random.randint(16)

    wires = []
   

    fh.write(f"// design #: {index_str}\n")
    fh.write(f"// num luts #: {NUM_LUTS}\n")
    fh.write(f"\n")
    fh.write(f"\n")


    fh.write(f"`timescale 1ns / 1ps\n")
    fh.write(f"\n")
    # fh.write(f"module generic_func(fpga_in_0, fpga_in_1, fpga_in_2, fpga_in_3")
    fh.write(f"module fpga_design(")
    for lut_input_idx in range(NUM_INPUTS):
        fh.write(f"fpga_in_{lut_input_idx}, ")
        # wires.append(Wire(f"fpga_in_{lut_input_idx}",-1,-1,"in"))
    for lut_idx in range(NUM_LUTS):
        if lut_idx < NUM_LUTS - 1:
            fh.write(f"fpga_out_{lut_idx}, ")
            # wires.append(Wire(f"fpga_out_{lut_idx}",-1,-1,"out"))
            
        else:
            fh.write(f"fpga_out_{lut_idx}")
    fh.write(f");\n")
    fh.write(f"\n")

    for j in range(NUM_INPUTS):
        fh.write(f"input wire fpga_in_{j};\n")

    for lut_idx in range(NUM_LUTS):
        fh.write(f"output reg fpga_out_{lut_idx};\n")


    # randomize connections between LUTs with wires
    for lut_idx in range(NUM_LUTS):

        ## declare LUT wire inputs
        fh.write(f"// LUT {lut_idx} Inputs \n")
        for lut_input_idx in range(NUM_LUT_INPUTS):
            in_wire_name = f"lut_{lut_idx}_in_{lut_input_idx}"
            fh.write(f"wire {in_wire_name};\n")
            wires.append(Wire(in_wire_name, lut_idx, lut_input_idx, "in"))

        ## declare LUT output
        out_wire_name = f"lut_{lut_idx}_out"
        fh.write(f"wire {out_wire_name};\n")

        ## assign LUT output to FPGA output
        fh.write(f"assign fpga_out_{lut_idx} = lut_{lut_idx}_out;\n")

        ## randomly assign lut inputs
        for lut_input_idx in range(NUM_LUT_INPUTS):
            lut_input_choice = random.randint(0,NUM_INPUTS - 1)
            fh.write(f"assign lut_{lut_idx}_in_{lut_input_idx} = fpga_in_{lut_input_choice};\n")


        fh.write(f"\n")


    for lut_idx in range(NUM_LUTS):
        fh.write(f"\n")
        fh.write(f"always @(*) begin\n")
        fh.write(f"\tcase ({{")
        for lut_input_idx in range(NUM_LUT_INPUTS-1,-1,-1):
            if lut_input_idx > 0:
                fh.write(f"lut_{lut_idx}_in_{lut_input_idx}, ")
            else:
                fh.write(f"lut_{lut_idx}_in_{lut_input_idx}")
        fh.write(f"}})\n")
        for j in range(2**NUM_LUT_INPUTS):
            fh.write(f"\t\t4'b{bin(j)[2:].zfill(4)}: lut_{lut_idx}_out <= {random.randint(0,1)};\n")
        fh.write(f"\t\tdefault: lut_{lut_idx}_out <= 0;\n")
        fh.write(f"\tendcase\n")
        fh.write(f"end\n")

    fh.write(f"endmodule\n")
    fh.close()

In [45]:
# write task.conf

NUM_DESIGNS=20000

fh = open(f"./results/random_designs/task.conf","w+")


fh.write(f"[GENERAL]\n")
fh.write(f"run_engine=openfpga_shell\n")
fh.write(f"power_tech_file = ${{PATH:OPENFPGA_PATH}}/openfpga_flow/tech/PTM_45nm/45nm.xml\n")
fh.write(f"power_analysis = true\n")
fh.write(f"spice_output=false\n")
fh.write(f"verilog_output=true\n")
fh.write(f"timeout_each_job = 20*60\n")
fh.write(f"fpga_flow=yosys_vpr\n")
fh.write(f"\n")

fh.write(f"[OpenFPGA_SHELL]\n")
fh.write(f"openfpga_shell_template=${{PATH:OPENFPGA_PATH}}/openfpga_flow/openfpga_shell_scripts/write_full_testbench_example_script.openfpga\n")
fh.write(f"openfpga_arch_file=${{PATH:OPENFPGA_PATH}}/openfpga_flow/openfpga_arch/k4_N4_40nm_cc_openfpga.xml\n")
fh.write(f"openfpga_sim_setting_file=${{PATH:OPENFPGA_PATH}}/openfpga_flow/openfpga_simulation_settings/auto_sim_openfpga.xml\n")
fh.write(f"openfpga_vpr_device_layout=--device 2x2 --route_chan_width 40\n")
fh.write(f"openfpga_fast_configuration=\n")
fh.write(f"\n")

fh.write(f"[ARCHITECTURES]\n")
fh.write(f"arch0=${{PATH:OPENFPGA_PATH}}/openfpga_flow/vpr_arch/k4_N4_tileable_40nm_new.xml\n")
fh.write(f"\n")

fh.write(f"[BENCHMARKS]\n")
for i in range(NUM_DESIGNS):
    design_num = f"{(i+1)}".zfill(5)
    fh.write(f"bench{i}=${{PATH:OPENFPGA_PATH}}/debug/architectures/arch_gen/results/random_designs/{design_num}/design.v\n")
fh.write(f"\n")

fh.write(f"[SYNTHESIS_PARAM]\n")
fh.write(f"bench_read_verilog_options_common = -nolatches\n")
fh.write(f"\n")

for i in range(NUM_DESIGNS):
    design_num = f"{(i+1)}".zfill(4)
    fh.write(f"bench{i}_top=fpga_design\n")
    fh.write(f"bench{i}_chan_width=16\n")
fh.write(f"\n")

fh.write(f"[SCRIPT_PARAM_MIN_ROUTE_CHAN_WIDTH]\n")
fh.write(f"end_flow_with_test=\n")


fh.close()

```
python3 openfpga_flow/scripts/run_fpga_task.py basic_tests/0_debug_task/random_designs --maxthreads=8 --debug
```

In [ ]:
import os

os.mkdir(f"/home/oshears/Documents/openfpga/OpenFPGA/openfpga_flow/tasks/basic_tests/1_random_designs/design_{design_num}")
os.mkdir(f"/home/oshears/Documents/openfpga/OpenFPGA/openfpga_flow/tasks/basic_tests/1_random_designs/design_{design_num}/config")

In [53]:
## copy bitstreams script

fh = open("/home/oshears/Documents/openfpga/OpenFPGA/debug/scripts/copy_bitstreams.sh","w+")

run_folder = "run014"
fh.write("rm -rf /home/oshears/Documents/openfpga/OpenFPGA/debug/architectures/random_bitstreams\n")
fh.write("mkdir /home/oshears/Documents/openfpga/OpenFPGA/debug/architectures/random_bitstreams\n")
for i in range(20000):
    idx = f"{i}".zfill(5)
    fh.write(f"cp /home/oshears/Documents/openfpga/OpenFPGA/openfpga_flow/tasks/basic_tests/0_debug_task/random_designs/{run_folder}/k4_N4_tileable_40nm_new/bench{i}_fpga_design/MIN_ROUTE_CHAN_WIDTH/fabric_bitstream.bit /home/oshears/Documents/openfpga/OpenFPGA/debug/architectures/random_bitstreams/bitstream_{idx}.bit\n")

fh.close()

## check bitstream lengths

In [54]:
import os

files = [f for f in os.listdir("../random_bitstreams/")]
files.sort()

lengths = {}
for file in files:
    fh = open(f"../random_bitstreams/{file}")
    
    fh.readline()
    fh.readline()
    fh.readline()
    lengthLine = fh.readline()
    lengthLine = lengthLine.strip()
    lengthLine = lengthLine.split(" ")
    length = lengthLine[-1]
    # print(lengthLine.strip())
    if length in lengths.keys():
        lengths[length] += 1
    else:
        lengths[length] = 1
        
    # if length != "1702":
    #     print(f"{file} : {length}")
    fh.close()
    
print(lengths)

{'1702': 20000}


In [32]:
import xml.etree.ElementTree as ET
tree = ET.parse('/home/oshears/Documents/openfpga/OpenFPGA/openfpga_flow/tasks/basic_tests/0_debug_task/random_designs/run006/k4_N4_tileable_40nm_new/bench0_fpga_design/MIN_ROUTE_CHAN_WIDTH/fabric_independent_bitstream.xml')
root = tree.getroot()

def recurseBitLength(root):

    total_bits = 0

    for child in root:
        total_bits += recurseBitLength(child)
    
    if "value" in root.attrib.keys():
        total_bits += 1
    
    return total_bits

total_bit_length = 0
bit_dict = {}
for child in root:
    # print(child.attrib)
    child_bit_length = recurseBitLength(child)
    # print(child_bit_length)
    bit_dict[child.attrib['name']] = child_bit_length
    print(f"Module: {child.attrib['name']}; Bits: {child_bit_length}")
    total_bit_length += child_bit_length



print(f"Total Bit Length: {total_bit_length}")


print(f"grid_io_bottom_1__0_: {bit_dict['grid_io_bottom_1__0_']}")
print(f"grid_io_bottom_2__0_: {bit_dict['grid_io_bottom_2__0_']}")
print(f"grid_io_right_3__1_: {bit_dict['grid_io_right_3__1_']}")
print(f"grid_io_right_3__2_: {bit_dict['grid_io_right_3__2_']}")
print(f"sb_2__2_: {bit_dict['sb_2__2_']}")
print(f"cbx_2__2_: {bit_dict['cbx_2__2_']}")
print(f"grid_io_top_2__3_: {bit_dict['grid_io_top_2__3_']}")
print(f"sb_1__2_: {bit_dict['sb_1__2_']}")
print(f"cbx_1__2_: {bit_dict['cbx_1__2_']}")
print(f"grid_io_top_1__3_: {bit_dict['grid_io_top_1__3_']}")
print(f"sb_0__2_: {bit_dict['sb_0__2_']}")
print(f"sb_0__1_: {bit_dict['sb_0__1_']}")
print(f"cby_0__2_: {bit_dict['cby_0__2_']}")
print(f"grid_io_left_0__2_: {bit_dict['grid_io_left_0__2_']}")
print(f"sb_0__0_: {bit_dict['sb_0__0_']}")
print(f"cby_0__1_: {bit_dict['cby_0__1_']}")
print(f"grid_io_left_0__1_: {bit_dict['grid_io_left_0__1_']}")
print(f"sb_1__0_: {bit_dict['sb_1__0_']}")
print(f"cbx_1__0_: {bit_dict['cbx_1__0_']}")
print(f"cby_1__1_: {bit_dict['cby_1__1_']}")
print(f"grid_clb_1__1_: {bit_dict['grid_clb_1__1_']}")
print(f"sb_2__0_: {bit_dict['sb_2__0_']}")
print(f"cbx_2__0_: {bit_dict['cbx_2__0_']}")
print(f"cby_2__1_: {bit_dict['cby_2__1_']}")
print(f"grid_clb_2__1_: {bit_dict['grid_clb_2__1_']}")
print(f"sb_2__1_: {bit_dict['sb_2__1_']}")
print(f"cbx_2__1_: {bit_dict['cbx_2__1_']}")
print(f"cby_2__2_: {bit_dict['cby_2__2_']}")
print(f"grid_clb_2__2_: {bit_dict['grid_clb_2__2_']}")
print(f"sb_1__1_: {bit_dict['sb_1__1_']}")
print(f"cbx_1__1_: {bit_dict['cbx_1__1_']}")
print(f"cby_1__2_: {bit_dict['cby_1__2_']}")
print(f"grid_clb_1__2_: {bit_dict['grid_clb_1__2_']}")


Module: grid_clb_1__1_; Bits: 136
Module: grid_clb_1__2_; Bits: 136
Module: grid_clb_2__1_; Bits: 136
Module: grid_clb_2__2_; Bits: 136
Module: grid_io_top_1__3_; Bits: 8
Module: grid_io_top_2__3_; Bits: 8
Module: grid_io_right_3__2_; Bits: 8
Module: grid_io_right_3__1_; Bits: 8
Module: grid_io_bottom_2__0_; Bits: 8
Module: grid_io_bottom_1__0_; Bits: 8
Module: grid_io_left_0__1_; Bits: 8
Module: grid_io_left_0__2_; Bits: 8
Module: sb_0__0_; Bits: 72
Module: sb_0__1_; Bits: 76
Module: sb_0__2_; Bits: 72
Module: sb_1__0_; Bits: 74
Module: sb_1__1_; Bits: 80
Module: sb_1__2_; Bits: 80
Module: sb_2__0_; Bits: 72
Module: sb_2__1_; Bits: 80
Module: sb_2__2_; Bits: 72
Module: cbx_1__0_; Bits: 40
Module: cbx_1__1_; Bits: 20
Module: cbx_1__2_; Bits: 44
Module: cbx_2__0_; Bits: 40
Module: cbx_2__1_; Bits: 20
Module: cbx_2__2_; Bits: 44
Module: cby_0__1_; Bits: 40
Module: cby_0__2_; Bits: 40
Module: cby_1__1_; Bits: 20
Module: cby_1__2_; Bits: 20
Module: cby_2__1_; Bits: 44
Module: cby_2__2_; Bi

## bitstream annotating

In [55]:
import os

bit_mapping = {"grid_io_bottom_1__0_": 8,
    "grid_io_bottom_2__0_": 8,
    "grid_io_right_3__1_": 8,
    "grid_io_right_3__2_": 8,
    "sb_2__2_": 72,
    "cbx_2__2_": 44,
    "grid_io_top_2__3_": 8,
    "sb_1__2_": 80,
    "cbx_1__2_": 44,
    "grid_io_top_1__3_": 8,
    "sb_0__2_": 72,
    "sb_0__1_": 76,
    "cby_0__2_": 40,
    "grid_io_left_0__2_": 8,
    "sb_0__0_": 72,
    "cby_0__1_": 40,
    "grid_io_left_0__1_": 8,
    "sb_1__0_": 74,
    "cbx_1__0_": 40,
    "cby_1__1_": 20,
    "grid_clb_1__1_": 136,
    "sb_2__0_": 72,
    "cbx_2__0_": 40,
    "cby_2__1_": 44,
    "grid_clb_2__1_": 136,
    "sb_2__1_": 80,
    "cbx_2__1_": 20,
    "cby_2__2_": 44,
    "grid_clb_2__2_": 136,
    "sb_1__1_": 80,
    "cbx_1__1_": 20,
    "cby_1__2_": 20,
    "grid_clb_1__2_": 136}

module_order = [
    "grid_io_bottom_1__0_",
    "grid_io_bottom_2__0_",
    "grid_io_right_3__1_",
    "grid_io_right_3__2_",
    "sb_2__2_",
    "cbx_2__2_",
    "grid_io_top_2__3_",
    "sb_1__2_",
    "cbx_1__2_",
    "grid_io_top_1__3_",
    "sb_0__2_",
    "sb_0__1_",
    "cby_0__2_",
    "grid_io_left_0__2_",
    "sb_0__0_",
    "cby_0__1_",
    "grid_io_left_0__1_",
    "sb_1__0_",
    "cbx_1__0_",
    "cby_1__1_",
    "grid_clb_1__1_",
    "sb_2__0_",
    "cbx_2__0_",
    "cby_2__1_",
    "grid_clb_2__1_",
    "sb_2__1_",
    "cbx_2__1_",
    "cby_2__2_",
    "grid_clb_2__2_",
    "sb_1__1_",
    "cbx_1__1_",
    "cby_1__2_",
    "grid_clb_1__2_"
]

files = [f for f in os.listdir("../random_bitstreams/")]
files.sort()

lengths = {}
for file in files:
    fh = open(f"../random_bitstreams/{file}")

    fh_csv = open(f"../random_bitstreams/{file}.csv","w+")
    fh_csv.write(f"module,type,value\n")
    
    # skip first 5 lines
    for i in range(5):
        fh.readline()

    module_idx = len(module_order) - 1
    module_bits = 0
    for idx in range(1702):
        bit_value = fh.readline().strip()
        if "grid_clb" in module_order[module_idx] and module_bits >= 64:
            fh_csv.write(f"{module_order[module_idx]},lut_data,{bit_value}\n")
        # elseif "grid_io" in module_order[module_idx]:
        #     fh_csv.write(f"{module_order[module_idx]},gpio_dir,{bit_value}\n")
        else:
            fh_csv.write(f"{module_order[module_idx]},routing,{bit_value}\n")
        module_bits += 1
        if module_bits == bit_mapping[module_order[module_idx]]:
            module_idx -= 1
            module_bits = 0

    fh.close()
    fh_csv.close()